Author: Mudit Soni

In [0]:
import pandas as pd
import numpy as np

# Read data and correct errors
data= pd.read_csv('TedoneItemAssignmentTable incomplete.csv')
test_data= pd.read_csv('Enneagram questions for test 23.11.19.csv')
bad_idx= data.index[data['Unnamed: 4']=='Achievement-striving'].tolist()
for idx in bad_idx:
    data.loc[idx, 'Unnamed: 4']= 'Achievement-Striving'

In [ ]:
# Preprocess the data
from sentence_embedder import *

embedder= SentenceEmbedder()
size= data.shape[0]
class_scores= {str(x): [] for x in range(1, 10)}
X_txt= []
X= []
sentiment= []

for category in data['Unnamed: 4'].unique():
    current_scores= {str(x): 0 for x in range(1, 10)}
    rows= data.index[data['Unnamed: 4']==category].tolist()
    first= True
    for row in rows:
        X_txt.append(data.loc[row, 'Unnamed: 3'])
        X.append(embedder.get(data.loc[row, 'Unnamed: 3']))
        sentiment.append(data.loc[row, 'Unnamed: 2'])
        if first==True:
            for j in [3, 2, 1, -1]:
                try:
                    for i in data.loc[row, str(j)].split(','):
                        current_scores[i]=j #if data.loc[row, 'Unnamed: 2']==1 else -j
                except: 
                    continue
            first= False
        for key in class_scores.keys():
            class_scores[key].append(current_scores[key] if data.loc[row, 'Unnamed: 2']==1 else -current_scores[key])

class_scores_mod= {}
for key in class_scores.keys():
    class_scores[key]= np.array(class_scores[key])
    class_scores_mod[key]= np.zeros((size, 7))
    class_scores_mod[key][np.arange(size), class_scores[key]+3]= 1

X= np.array(X)
X_txt= np.array(X_txt)
sentiment= np.array(sentiment)
sents= np.vstack((sentiment==1, sentiment==-1)).astype(float).T

# Below is the model for classifying negative and positive sentences.

In [0]:
def classifier_model():
    inp = Input(shape=(512,))
    x = Dense(64, activation='relu')(inp)
    outp = Dense(2, activation='sigmoid')(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(optimizer='adam', loss= 'categorical_crossentropy', 
                    metrics=['accuracy'])
    return model

In [0]:
# Prepare the data.
from sklearn.model_selection import train_test_split

x_sent_tr, x_sent_te, sent_tr, sent_te= train_test_split(X, sents, test_size=0.1)

In [26]:
# Train model.
mod= classifier_model()
mod.fit(x_sent_tr, sent_tr, validation_data=(x_sent_te, sent_te), batch_size= 32, epochs= 200)

Train on 1020 samples, validate on 114 samples
Epoch 1/200
1020/1020 [==============================] - 0s 145us/sample - loss: 0.6696 - acc: 0.6049 - val_loss: 0.6767 - val_acc: 0.5526
Epoch 2/200
1020/1020 [==============================] - 0s 68us/sample - loss: 0.6373 - acc: 0.6147 - val_loss: 0.6601 - val_acc: 0.5526
Epoch 3/200
1020/1020 [==============================] - 0s 64us/sample - loss: 0.6044 - acc: 0.6627 - val_loss: 0.6394 - val_acc: 0.6140
Epoch 4/200
1020/1020 [==============================] - 0s 54us/sample - loss: 0.5750 - acc: 0.6912 - val_loss: 0.6329 - val_acc: 0.6316
Epoch 5/200
1020/1020 [==============================] - 0s 56us/sample - loss: 0.5477 - acc: 0.7265 - val_loss: 0.6341 - val_acc: 0.6140
Epoch 6/200
1020/1020 [==============================] - 0s 56us/sample - loss: 0.5260 - acc: 0.7471 - val_loss: 0.6277 - val_acc: 0.6053
Epoch 7/200
1020/1020 [==============================] - 0s 62us/sample - loss: 0.5110 - acc: 0.7461 - val_loss: 0.6466 - va

# Below are the models for generating class scores for sentences.

Custom loss functions:

In [0]:
import tensorflow as tf
from tensorflow.keras.metrics import Metric
import tensorflow.keras.backend as K
RANDOM_SEED = 3
tf.random.set_seed(RANDOM_SEED)
tf.keras.backend.set_floatx('float64')

@tf.function
def cohen_kappa_loss(y_true, y_pred, row_label_vec, col_label_vec, weight_mat,  eps=1e-6, dtype=tf.float64):
    labels = tf.matmul(y_true, col_label_vec)
    weight = tf.pow(tf.tile(labels, [1, tf.shape(y_true)[1]]) - tf.tile(row_label_vec, [tf.shape(y_true)[0], 1]), 2)
    weight /= tf.cast(tf.pow(tf.shape(y_true)[1] - 1, 2), dtype=dtype)
    numerator = tf.reduce_sum(weight * y_pred)
    
    denominator = tf.reduce_sum(
        tf.matmul(
            tf.reduce_sum(y_true, axis=0, keepdims=True),
            tf.matmul(weight_mat, tf.transpose(tf.reduce_sum(y_pred, axis=0, keepdims=True)))
        )
    )
    
    denominator /= tf.cast(tf.shape(y_true)[0], dtype=dtype)
    
    return tf.math.log(numerator / denominator + eps)

class CohenKappaLoss(tf.keras.losses.Loss):
    def __init__(self,
                 num_classes,
                 name='cohen_kappa_loss',
                 eps=1e-6,
                 dtype=tf.float64):
        super(CohenKappaLoss, self).__init__(name=name, reduction=tf.keras.losses.Reduction.NONE)
        
        self.num_classes = num_classes
        self.eps = eps
        self.dtype = dtype
        label_vec = tf.range(num_classes, dtype=dtype)
        self.row_label_vec = tf.reshape(label_vec, [1, num_classes])
        self.col_label_vec = tf.reshape(label_vec, [num_classes, 1])
        self.weight_mat = tf.pow(
            tf.tile(self.col_label_vec, [1, num_classes]) - tf.tile(self.row_label_vec, [num_classes, 1]),
        2) / tf.cast(tf.pow(num_classes - 1, 2), dtype=dtype)


    def call(self, y_true, y_pred, sample_weight=None):
        return cohen_kappa_loss(
            y_true, y_pred, self.row_label_vec, self.col_label_vec, self.weight_mat, self.eps, self.dtype
        )


    def get_config(self):
        config = {
            "num_classes": self.num_classes,
            "eps": self.eps,
            "dtype": self.dtype
        }
        base_config = super(CohenKappaLoss, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
class CohenKappa(Metric):
    """
    This metric is copied from TensorFlow Addons
    """
    def __init__(self,
                 num_classes,
                 name='cohen_kappa',
                 weightage=None,
                 dtype=tf.float32):
        super(CohenKappa, self).__init__(name=name, dtype=dtype)

        if weightage not in (None, 'linear', 'quadratic'):
            raise ValueError("Unknown kappa weighting type.")
        else:
            self.weightage = weightage

        self.num_classes = num_classes
        self.conf_mtx = self.add_weight(
            'conf_mtx',
            shape=(self.num_classes, self.num_classes),
            initializer=tf.keras.initializers.zeros,
            dtype=tf.int32)
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        if len(y_true.shape) == 2:
            y_true = tf.argmax(y_true, axis=1)
        if len(y_pred.shape) == 2:
            y_pred = tf.argmax(y_pred, axis=1)
        
        y_true = tf.cast(y_true, dtype=tf.int32)
        y_pred = tf.cast(y_pred, dtype=tf.int32)
        
        if y_true.shape.as_list() != y_pred.shape.as_list():
            raise ValueError(
                "Number of samples in y_true and y_pred are different")

        # compute the new values of the confusion matrix
        new_conf_mtx = tf.math.confusion_matrix(
            labels=y_true,
            predictions=y_pred,
            num_classes=self.num_classes,
            weights=sample_weight)

        # update the values in the original confusion matrix
        return self.conf_mtx.assign_add(new_conf_mtx)
    
    def result(self):
        nb_ratings = tf.shape(self.conf_mtx)[0]
        weight_mtx = tf.ones([nb_ratings, nb_ratings], dtype=tf.int32)

        # 2. Create a weight matrix
        if self.weightage is None:
            diagonal = tf.zeros([nb_ratings], dtype=tf.int32)
            weight_mtx = tf.linalg.set_diag(weight_mtx, diagonal=diagonal)
            weight_mtx = tf.cast(weight_mtx, dtype=tf.float32)

        else:
            weight_mtx += tf.range(nb_ratings, dtype=tf.int32)
            weight_mtx = tf.cast(weight_mtx, dtype=tf.float32)

            if self.weightage == 'linear':
                weight_mtx = tf.abs(weight_mtx - tf.transpose(weight_mtx))
            else:
                weight_mtx = tf.pow((weight_mtx - tf.transpose(weight_mtx)), 2)
            weight_mtx = tf.cast(weight_mtx, dtype=tf.float32)

        # 3. Get counts
        actual_ratings_hist = tf.reduce_sum(self.conf_mtx, axis=1)
        pred_ratings_hist = tf.reduce_sum(self.conf_mtx, axis=0)

        # 4. Get the outer product
        out_prod = pred_ratings_hist[..., None] * \
                    actual_ratings_hist[None, ...]

        # 5. Normalize the confusion matrix and outer product
        conf_mtx = self.conf_mtx / tf.reduce_sum(self.conf_mtx)
        out_prod = out_prod / tf.reduce_sum(out_prod)

        conf_mtx = tf.cast(conf_mtx, dtype=tf.float32)
        out_prod = tf.cast(out_prod, dtype=tf.float32)

        # 6. Calculate Kappa score
        numerator = tf.reduce_sum(conf_mtx * weight_mtx)
        denominator = tf.reduce_sum(out_prod * weight_mtx)
        kp = 1 - (numerator / denominator)
        return kp
    
    def get_config(self):
        """Returns the serializable config of the metric."""

        config = {
            "num_classes": self.num_classes,
            "weightage": self.weightage,
        }
        base_config = super(CohenKappa, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def reset_states(self):
        """Resets all of the metric state variables."""

        for v in self.variables:
            K.set_value(
                v, np.zeros((self.num_classes, self.num_classes), np.int32))

In [0]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

The ordinal regression model:

In [0]:
import tensorflow.compat.v2.keras as keras
from tensorflow.compat.v2.keras.layers import Dense, Input, Dropout
from tensorflow.compat.v2.keras.models import Model

def build_model1():
    inp = Input(shape=(512,))
    x = Dense(128, activation='relu')(inp)
    #x= Dropout(0.5)(x)
    #x= Dense(32, activation='relu')(x)
    outp = Dense(7, activation='softmax')(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(optimizer='adam', loss=CohenKappaLoss(7), 
                  metrics=[CohenKappa(num_classes=7, weightage='quadratic')])
    return model

In [0]:
# Prepare the data.
mask = np.random.rand(size) < 0.9
y_train, y_test= {}, {}

X_train, X_test, X_txt_train, X_txt_test = X[mask], X[~mask], X_txt[mask], X_txt[~mask]
for key in class_scores_mod.keys():
    y_train[key], y_test[key]= class_scores_mod[key][mask], class_scores_mod[key][~mask]

In [ ]:
# Train models.
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor = 'val_cohen_kappa', min_delta = 0, patience = 500, mode='max', restore_best_weights=True)

models= {}
for param in list(class_scores_mod.keys()):
    model = build_model1()
    model.fit(X_train, y_train[param], validation_data=(X_test,y_test[param]), callbacks= [earlystop], batch_size=64, epochs=500)
    models[param]= model

Results: (Can be improved by babysitting training for individual models.)

In [0]:
from sklearn.metrics import confusion_matrix

for param in list(class_scores_mod.keys()):
    print('Training CM '+param)
    y_pred = models[param].predict(X_train)
    cm = confusion_matrix(y_train[param].argmax(axis=1), y_pred.argmax(axis=1))
    print(cm) 
    print('Testing CM '+param)
    y_pred = models[param].predict(X_test)
    cm = confusion_matrix(y_test[param].argmax(axis=1), y_pred.argmax(axis=1))
    print(cm) 

Training CM 1
[[  0   0  62   0   0   0   0]
 [  0   0  12   3   0   1   0]
 [  0   0 130  48   0   2   0]
 [  0   0  27 479   0  11   3]
 [  0   0   2  79   0  53   7]
 [  0   0   0   1   0   8  19]
 [  0   0   0   3   0  11  70]]
Testing CM 1
[[ 0  0  3  2  0  0  1]
 [ 0  0  0  1  0  1  0]
 [ 0  0  7 11  0  1  0]
 [ 0  0  4 43  0  2  2]
 [ 0  0  2 10  0  7  1]
 [ 0  0  0  1  0  0  0]
 [ 0  0  2  0  0  1  1]]
Training CM 2
[[  0  21   0   4   0   0   1]
 [  0  29   0  15   0   0   1]
 [  0  18   0  20   0   0   0]
 [  0   7   0 772   0   0   5]
 [  0   0   0  12   0   0   2]
 [  0   0   0  14   0   0  57]
 [  0   1   0   4   0   0  48]]
Testing CM 2
[[ 0  3  0  2  0  0  0]
 [ 0  1  0  2  0  0  0]
 [ 0  0  0  3  0  0  1]
 [ 0  0  0 74  0  0  4]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0  4  0  0  4]
 [ 0  0  0  2  0  0  2]]
Training CM 3
[[  0  34   0   6   0   0   0]
 [  0   4   0   5   0   0   0]
 [  0  56   0  42   0   0   0]
 [  0   7   0 655  19   0   4]
 [  0   0   0  41  66   0  10]
 [ 

Test on custom sentences:

In [0]:
sent1= 'You Expand your energy, express your desires and anger'
sent2= 'Expand your energy, express your desires and anger'

x1= embedder.get(sent1).reshape((-1, 512))
x2= embedder.get(sent2).reshape((-1, 512))

print(sent1)
for param in list(class_scores_mod.keys()):
    #maxim= models[param].predict(x1).max(axis=1)
    pred= np.argmax(models[param].predict(x1))-3
    print(param, pred)

print(sent2)
for param in list(class_scores_mod.keys()):
    #maxim= models[param].predict(x2).max(axis=1)
    pred= np.argmax(models[param].predict(x2))-3
    print(param, pred)

You Expand your energy, express your desires and anger
1 2
2 0
3 0
4 0
5 0
6 0
7 0
8 3
9 0
Expand your energy, express your desires and anger
1 2
2 0
3 0
4 0
5 0
6 0
7 0
8 3
9 0


Test on test dataset:

In [0]:
print(test_data.head)

<bound method NDFrame.head of      Unnamed: 0                                         Unnamed: 1  ...    8    9
0           NaN                                                NaN  ...  NaN  NaN
1           NaN  While addressing others' needs you are: Inflex...  ...  NaN  NaN
2           NaN  While addressing others' needs you are: Flexib...  ...  NaN  NaN
3           NaN                                                NaN  ...  NaN  NaN
4           NaN                                                NaN  ...  NaN  NaN
..          ...                                                ...  ...  ...  ...
426         NaN  You Focus on others opinions or agendas and ca...  ...  NaN    #
427         NaN                                                NaN  ...  NaN  NaN
428         NaN                                                NaN  ...  NaN  NaN
429         NaN    You Are clear about your views and are decisive  ...    #  NaN
430         NaN  You Are unclear about your opinions, indecisiv...  

In [0]:
# Preprocess the data.
test_sent= []
test_target= []
test_embds= []
test_scores= {str(x): [] for x in range(1, 10)}

for i in range(test_data.shape[0]):
    if type(test_data.loc[i, 'Unnamed: 1'])==str:
        sent= test_data.loc[i, 'Unnamed: 1']
        test_sent.append(sent)
        sent= sent.split(':')[-1].split('You ')[-1].strip()
        print(sent)
        embd= embedder.get(sent)
        test_embds.append(embd)
        test_target.append(test_data.loc[i].index[test_data.loc[i]=='#'][0])

        for param in list(test_scores.keys()):
            #maxim= models[param].predict(x2).max(axis=1)
            pred= np.argmax(models[param].predict(embd.reshape((-1, 512))))-3
            test_scores[param].append(pred)

Inflexible and stick to your needs
Flexible and alter your needs to make others happy
Completely independent of others
Interdependent and connected with others
Overcome your personal terms and follow principles to do what is right
Prefer personal terms, sometimes not going by principles
Inner critic to do what is judged as right
Drive to succeed and be recognized for your accomplishments
find the method to do it perfectly and stick to it until it succeeds
find the most efficient/ fastest possible method and in case it doesnot work , you change the method quickly to achieve the desired result
tend to express your emotions openly and people are easily aware of it
are not usually distracted by your feelings without people being aware of your emotional state
tend to Suppress personal desires and become self constrained
tend to Experience strong personal desires to the point of being self-absorbed (disconnected from outside world)
Focus on responsibilities before emotions
Sort out emotions 

In [0]:
results= pd.DataFrame(test_sent)
results= results.join(pd.DataFrame(test_scores))
results.to_csv('enneagram_results.csv')

In [0]:
# Get top3 scores.
top3= []

for i in range(results.shape[0]):
    inds= results.loc[i].index[results.loc[i]==3].tolist()
    if len(inds)==0:
        inds= results.loc[i].index[results.loc[i]==-3].tolist()
        inds= inds+results.loc[i].index[results.loc[i]==-2].tolist()
        inds= inds+results.loc[i].index[results.loc[i]==-1].tolist()+['0', '0', '0']
    else:
        inds= inds+results.loc[i].index[results.loc[i]==-2].tolist()
        inds= inds+results.loc[i].index[results.loc[i]==-1].tolist()+['0', '0', '0']
    inds= inds[:3]
    top3.append(inds)

top3= np.asarray(top3)
print(top3)

[['3' '8' '9']
 ['2' '9' '0']
 ['9' '6' '8']
 ['2' '9' '0']
 ['1' '7' '0']
 ['7' '0' '0']
 ['0' '0' '0']
 ['3' '8' '9']
 ['1' '5' '8']
 ['1' '3' '5']
 ['4' '2' '0']
 ['4' '2' '0']
 ['2' '6' '0']
 ['4' '0' '0']
 ['0' '0' '0']
 ['8' '1' '5']
 ['1' '5' '0']
 ['2' '0' '0']
 ['2' '4' '0']
 ['1' '2' '0']
 ['1' '0' '0']
 ['0' '0' '0']
 ['4' '1' '5']
 ['1' '0' '0']
 ['4' '0' '0']
 ['1' '5' '6']
 ['1' '5' '6']
 ['5' '6' '7']
 ['2' '8' '0']
 ['4' '2' '8']
 ['2' '0' '0']
 ['7' '2' '0']
 ['2' '5' '0']
 ['4' '5' '7']
 ['8' '0' '0']
 ['1' '8' '3']
 ['9' '1' '2']
 ['9' '8' '0']
 ['1' '5' '9']
 ['8' '0' '0']
 ['1' '7' '0']
 ['1' '0' '0']
 ['5' '7' '0']
 ['8' '0' '0']
 ['1' '9' '0']
 ['2' '0' '0']
 ['1' '7' '9']
 ['1' '7' '0']
 ['2' '0' '0']
 ['3' '8' '0']
 ['2' '0' '0']
 ['6' '2' '0']
 ['1' '2' '0']
 ['2' '1' '0']
 ['1' '5' '7']
 ['9' '6' '8']
 ['9' '2' '0']
 ['9' '2' '8']
 ['2' '0' '0']
 ['2' '0' '0']
 ['4' '0' '0']
 ['2' '0' '0']
 ['6' '2' '7']
 ['6' '2' '7']
 ['2' '4' '9']
 ['4' '5' '0']
 ['2' '0' 

In [0]:
top3_score= 0
top2_score= 0
top1_score= 0

for i in range(top3.shape[0]):
    if test_target[i] in top3[i]:
        top3_score+= 1
    if test_target[i] in top3[i, :2]:
        top2_score+= 1
    if test_target[i] == top3[i, 0]:
        top1_score+= 1

top3_score/= 216
top2_score/= 216
top1_score/= 216

print(top3_score)

0.39814814814814814
